In [9]:
%load_ext autoreload
%autoreload 2

from sanity_check.traffic_generation import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
scenarios: List[Scenario] = []
instances: Dict[Scenario, List[TrafficGenerator]] = {}
instance_paths: List[str] = glob(os.path.join("/storage/research_data/sommer2020cadr/maci-docker-compose/maci_data//binary_files/6/ids/", "*"))

In [34]:
instance_paths

['/storage/research_data/sommer2020cadr/maci-docker-compose/maci_data//binary_files/6/ids/10',
 '/storage/research_data/sommer2020cadr/maci-docker-compose/maci_data//binary_files/6/ids/28',
 '/storage/research_data/sommer2020cadr/maci-docker-compose/maci_data//binary_files/6/ids/55',
 '/storage/research_data/sommer2020cadr/maci-docker-compose/maci_data//binary_files/6/ids/22',
 '/storage/research_data/sommer2020cadr/maci-docker-compose/maci_data//binary_files/6/ids/46',
 '/storage/research_data/sommer2020cadr/maci-docker-compose/maci_data//binary_files/6/ids/31',
 '/storage/research_data/sommer2020cadr/maci-docker-compose/maci_data//binary_files/6/ids/36',
 '/storage/research_data/sommer2020cadr/maci-docker-compose/maci_data//binary_files/6/ids/41',
 '/storage/research_data/sommer2020cadr/maci-docker-compose/maci_data//binary_files/6/ids/58',
 '/storage/research_data/sommer2020cadr/maci-docker-compose/maci_data//binary_files/6/ids/25',
 '/storage/research_data/sommer2020cadr/maci-docke

In [35]:
def parse_instances() :
    for instance_path in instance_paths:
        scenario = parse_instance_parameters(os.path.join(instance_path, "parameters.py"))
        scenarios.append(scenario)

        node_paths = glob(os.path.join(instance_path, "*.conf_traffic_generator_run.log"))
        generators: List[TrafficGenerator] = []
        for node_path in node_paths:
            node_generator = parse_traffic_generator(node_path)
            generators.append(node_generator)

        instances[scenario] = generators

parse_instances()

In [36]:
def run_stats():
    epidemic_should = 0
    epidemic_sent = 0
    epidemic_failed_run = 0
    epidemic_ok_run = 0
    epidemic_really_sent = 0

    cadr_should = 0
    cadr_sent = 0
    cadr_failed_run = 0
    cadr_ok_run = 0
    cadr_really_sent = 0
    
    for scenario in scenarios:
        for generator in instances[scenario]:
            if scenario.routing == "epidemic":
                epidemic_should = epidemic_should + generator.num_dtnd
                epidemic_sent = epidemic_sent + generator.num_sent
                epidemic_really_sent = epidemic_really_sent + generator.really_sent
                if generator.num_sent != generator.num_dtnd:
                    epidemic_failed_run = epidemic_failed_run + 1
                else:
                    epidemic_ok_run = epidemic_ok_run + 1
            else:
                cadr_should = cadr_should + generator.num_dtnd
                cadr_sent = cadr_sent + generator.num_sent
                cadr_really_sent = cadr_really_sent + generator.really_sent
                if generator.num_sent != generator.num_dtnd:
                    cadr_failed_run = cadr_failed_run + 1
                else:
                    cadr_ok_run = cadr_ok_run + 1

    print(f"Epidemic")
    print(f"  Generator should send {epidemic_sent} bundles, but actually sent {epidemic_really_sent} bundles.")
    print(f"  DTND received {epidemic_should} bundles, {epidemic_sent - epidemic_should}) missing.")
    print(f"  {epidemic_ok_run} nodes successful, {epidemic_failed_run} failed ({(epidemic_failed_run/(epidemic_failed_run + epidemic_ok_run)) * 100}%)")
    
    #print(f"CADR")
    #print(f"  Generator should send {cadr_sent} bundles, but actually sent {cadr_really_sent} bundles.")
    #print(f"  DTND received {cadr_should} bundles, {cadr_sent - cadr_should}) missing.")
    #print(f"  {cadr_ok_run} nodes successful, {cadr_failed_run} failed ({(cadr_failed_run/(cadr_failed_run + cadr_ok_run)) * 100}%)")
run_stats()

Epidemic
  Generator should send 12480 bundles, but actually sent 9705 bundles.
  DTND received 9706 bundles, 2774) missing.
  194 nodes successful, 85 failed (30.46594982078853%)


In [37]:
for scenario in scenarios:
    for other_scenario in scenarios:
        if scenario.instance_id == other_scenario.instance_id:
            continue
        if scenario.matches(other_scenario):
            generators_a = instances[scenario]
            generators_b = instances[other_scenario]

            for generator in generators_a:
                if generator.num_sent != generator.really_sent:
                    print(f"Mismatch: {scenario.instance_id}/{generator.node_id}({scenario.routing})")
                    print(f"  {generator.num_sent}, {generator.really_sent}")
                if generator.really_sent != generator.num_dtnd:
                    print(f"    {generator.num_dtnd}")
                
                for other_generator in generators_b:
                    if generator == other_generator:
                        if not generator.matches(other_generator):
                            foo = generator.matches(other_generator)
                            print(
                                f"#### FAIL: Mismatch between instances {scenario.instance_id} and {other_scenario.instance_id}:"
                            )
                            print(
                                f"#### FAIL: Params: bundles_per_node: {scenario.bundles_per_node}, seed: {scenario.seed}, payload_size: {scenario.payload_size}"
                            )
                            print(f"#### FAIL: Node: {generator.node_id}")
                            print("#### FAIL: Differences:")
                            generator.difference(other_generator)
                            print("#### FAIL: -----------------------------")

Mismatch: 28/n30(context_spray)
  10, 0
    10
Mismatch: 28/n24(context_spray)
  10, 0
    10
Mismatch: 28/n2(context_spray)
  10, 0
    10
Mismatch: 28/n20(context_spray)
  10, 0
    10
Mismatch: 28/n18(context_spray)
  10, 0
    10
Mismatch: 28/n14(context_spray)
  10, 0
    10
Mismatch: 28/n10(context_spray)
  10, 0
    10
Mismatch: 28/n28(context_spray)
  10, 0
    10
Mismatch: 28/n15(context_spray)
  10, 0
    10
Mismatch: 28/n11(context_spray)
  10, 0
    9
Mismatch: 28/n29(context_spray)
  10, 0
    10
Mismatch: 28/n31(context_spray)
  10, 0
    10
Mismatch: 28/n25(context_spray)
  10, 0
    10
Mismatch: 28/n21(context_spray)
  10, 0
    10
Mismatch: 28/n19(context_spray)
  10, 0
    10
Mismatch: 28/n17(context_spray)
  10, 0
    10
Mismatch: 28/n9(context_spray)
  10, 0
    10
Mismatch: 28/n13(context_spray)
  10, 0
    10
Mismatch: 28/n27(context_spray)
  10, 0
    10
Mismatch: 28/n23(context_spray)
  10, 0
    10
Mismatch: 28/n32(context_spray)
  10, 0
    10
Mismatch: 28/n26

In [38]:
def wait_time_diffs():
    for scenario in scenarios:
        for generator in instances[scenario]:
            for wait_time in generator.real_wait_times:
                if int(wait_time[1]) - wait_time[0] != 0:
                    print(f"Mismatch: {scenario}")

wait_time_diffs()

Mismatch: Scenario(bundles_per_node=10, seed=0, routing='dtlsr', payload_size=1000, instance_id='10')
Mismatch: Scenario(bundles_per_node=10, seed=0, routing='dtlsr', payload_size=1000, instance_id='10')
Mismatch: Scenario(bundles_per_node=10, seed=0, routing='dtlsr', payload_size=1000, instance_id='10')
Mismatch: Scenario(bundles_per_node=10, seed=0, routing='dtlsr', payload_size=1000, instance_id='10')
Mismatch: Scenario(bundles_per_node=10, seed=0, routing='dtlsr', payload_size=1000, instance_id='10')
Mismatch: Scenario(bundles_per_node=10, seed=0, routing='dtlsr', payload_size=1000, instance_id='10')
Mismatch: Scenario(bundles_per_node=10, seed=0, routing='context_spray', payload_size=1000, instance_id='28')
Mismatch: Scenario(bundles_per_node=10, seed=0, routing='context_spray', payload_size=1000, instance_id='28')
Mismatch: Scenario(bundles_per_node=10, seed=0, routing='context_spray', payload_size=1000, instance_id='28')
Mismatch: Scenario(bundles_per_node=10, seed=0, routing='c

In [41]:
for scenario in scenarios:
    for generator in instances[scenario]:
        if generator.node_id in ["n3", "n4", "n5", "n6", "n7"]:
            continue
        if(generator.bundles_per_node != generator.really_sent):
            print(f"Discrepancy: Should send {generator.bundles_per_node}, did sent: {generator.really_sent}")
            print(f"Length wait times: {len(generator.wait_times)}")
            print(f"Routing: {generator.routing}, Seed: {generator.seed}, Node ID: {generator.node_id}")
            print(f"Instance: {scenario.instance_id}")
            print(f"Soll: {generator.timestamps}") 
            print(f"ist:  {generator.sent_times}")
            print("----------------------------")

Discrepancy: Should send 10, did sent: 0
Length wait times: 10
Routing: context_spray, Seed: 0, Node ID: n30
Instance: 28
Soll: [238, 506, 847, 1204, 1529, 1882, 2282, 2739, 2906, 3370]
ist:  []
----------------------------
Discrepancy: Should send 10, did sent: 0
Length wait times: 10
Routing: context_spray, Seed: 0, Node ID: n24
Instance: 28
Soll: [94, 395, 1051, 1195, 1775, 1892, 2380, 2715, 2935, 3365]
ist:  []
----------------------------
Discrepancy: Should send 10, did sent: 0
Length wait times: 10
Routing: context_spray, Seed: 0, Node ID: n2
Instance: 28
Soll: [244, 630, 1005, 1204, 1808, 2170, 2365, 2552, 2899, 3509]
ist:  []
----------------------------
Discrepancy: Should send 10, did sent: 0
Length wait times: 10
Routing: context_spray, Seed: 0, Node ID: n20
Instance: 28
Soll: [105, 544, 1030, 1146, 1641, 2068, 2331, 2625, 2970, 3543]
ist:  []
----------------------------
Discrepancy: Should send 10, did sent: 0
Length wait times: 10
Routing: context_spray, Seed: 0, Node ID

In [32]:
start_times = {}
for scenario in scenarios:
    generator_start_times = []
    for generator in instances[scenario]:
        generator_start_times.append(generator.start_time)
    start_times[scenario.instance_id] = sorted(generator_start_times)
    
for t in start_times:
    print(start_times[t])

[1643903659, 1643903659, 1643903659, 1643903661, 1643903661, 1643903662, 1643903662, 1643903663, 1643903663, 1643903663, 1643903664, 1643903664, 1643903664, 1643903664, 1643903665, 1643903665, 1643903666, 1643903666, 1643903666, 1643903666, 1643903667, 1643903667, 1643903667, 1643903668, 1643903668, 1643903668, 1643903668, 1643903668, 1643903669, 1643903669, 1643903669]
[1643895498, 1643895499, 1643895500, 1643895501, 1643895501, 1643895501, 1643895502, 1643895502, 1643895502, 1643895502, 1643895503, 1643895503, 1643895504, 1643895504, 1643895504, 1643895504, 1643895504, 1643895504, 1643895506, 1643895506, 1643895506, 1643895506, 1643895506, 1643895507, 1643895507, 1643895507, 1643895507, 1643895507, 1643895507, 1643895508, 1643895508]
[1643907318, 1643907318, 1643907319, 1643907319, 1643907320, 1643907320, 1643907320, 1643907321, 1643907321, 1643907321, 1643907322, 1643907323, 1643907323, 1643907323, 1643907323, 1643907324, 1643907324, 1643907325, 1643907325, 1643907326, 1643907326, 1